In [1]:
%cd ~/REVIVAL2

/disk2/fli/REVIVAL2


In [2]:
%load_ext autoreload
%autoreload 2

In [18]:
# vis based on different metrics 
import numpy as np
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt
from scipy.stats import spearmanr

import warnings

from REVIVAL.util import ndcg, ndcg_scale, custom_ndcg, calc_top_n_percent_recall, get_file_name

# Suppress only ConstantInputWarning
warnings.simplefilter("ignore", category=UserWarning)


In [9]:
df = pd.read_csv("/disk2/fli/REVIVAL2/zs/comb/ParLQ.csv")
df

,AAs,var,fitness,selectivity,n_mut,enzyme,hd,AA1,AA2,AA3,...,vina_max_chai-score_joint-substrate_cofactor-docked,vina_chai-score_joint-substrate+carbene_cofactor-docked,vina_min_chai-score_joint-substrate+carbene_cofactor-docked,vina_max_chai-score_joint-substrate+carbene_cofactor-docked,vina_chai-score_joint-substrate_cofactor-score_only,vina_min_chai-score_joint-substrate_cofactor-score_only,vina_max_chai-score_joint-substrate_cofactor-score_only,vina_chai-score_joint-substrate+carbene_cofactor-score_only,vina_min_chai-score_joint-substrate+carbene_cofactor-score_only,vina_max_chai-score_joint-substrate+carbene_cofactor-score_only
0,WYLQA,F89A,0.000000,0.000000,1,ParLQ,-1,W,Y,L,...,-3.848,-8.5214,-9.536,-6.633,3.91520,-4.927,14.588,5.7420,-2.362,15.466
1,WYLQG,F89G,0.000000,0.000000,1,ParLQ,-1,W,Y,L,...,-4.556,-8.5296,-9.476,-7.296,-4.03220,-5.764,-2.030,-1.4432,-4.318,0.829
2,WYLQS,F89S,0.000000,0.000000,1,ParLQ,-1,W,Y,L,...,-4.553,-9.4034,-9.599,-8.868,-3.52500,-4.414,-2.384,-1.5294,-1.855,-1.296
3,QYRGL,W56Q:L59R:Q60G:F89L,0.000000,0.000000,4,ParLQ,-4,Q,Y,R,...,-4.386,-8.9612,-9.222,-8.550,-2.50160,-3.968,-0.504,-0.6146,-1.928,1.117
4,SYLQF,W56S,0.000000,0.000000,1,ParLQ,-1,S,Y,L,...,-4.029,-7.7448,-8.529,-5.461,1.24200,-1.569,4.893,3.5916,0.262,6.021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
485,MKFDY,W56M:Y57K:L59F:Q60D:F89Y,71.667516,12.444962,5,ParLQ,-5,M,K,F,...,-4.018,-7.6768,-9.533,-5.418,-0.40700,-4.995,4.204,1.3190,-3.985,7.447
486,MPFDY,W56M:Y57P:L59F:Q60D:F89Y,92.656383,14.026378,5,ParLQ,-5,M,P,F,...,-3.357,-8.8862,-9.719,-7.440,2.31540,-4.068,8.352,3.5910,-1.568,8.867
487,MAFDY,W56M:Y57A:L59F:Q60D:F89Y,69.699624,14.655944,5,ParLQ,-5,M,A,F,...,-5.565,-9.2682,-9.651,-8.281,-4.36020,-4.845,-3.975,-2.4870,-3.999,-0.348
488,MGFDY,W56M:Y57G:L59F:Q60D:F89Y,83.627237,20.159964,5,ParLQ,-5,M,G,F,...,-4.191,-8.3474,-9.546,-5.630,-0.08880,-4.185,3.152,1.6528,-2.218,4.888


In [10]:
df.select_dtypes(include=['number']).columns


Index(['fitness', 'selectivity', 'n_mut', 'hd', 'ev_score', 'esm_score',
       'flowsite_score', 'flowsite_std', 'ligandmpnn_score', 'var_vol',
       ...
       'vina_max_chai-score_joint-substrate_cofactor-docked',
       'vina_chai-score_joint-substrate+carbene_cofactor-docked',
       'vina_min_chai-score_joint-substrate+carbene_cofactor-docked',
       'vina_max_chai-score_joint-substrate+carbene_cofactor-docked',
       'vina_chai-score_joint-substrate_cofactor-score_only',
       'vina_min_chai-score_joint-substrate_cofactor-score_only',
       'vina_max_chai-score_joint-substrate_cofactor-score_only',
       'vina_chai-score_joint-substrate+carbene_cofactor-score_only',
       'vina_min_chai-score_joint-substrate+carbene_cofactor-score_only',
       'vina_max_chai-score_joint-substrate+carbene_cofactor-score_only'],
      dtype='object', length=516)

In [20]:
rho_list = []
ndcg_list = []
top10_list = []
top25_list = []
top50_list = []

for f in glob("/disk2/fli/REVIVAL2/zs/comb/*.csv"):
    df_original = pd.read_csv(f)
    df_original = df_original[df_original.select_dtypes(include=['number']).columns]
    df_name = get_file_name(f)
    print(df_name)
    
    rho_dict = {"lib": df_name}
    ndcg_dict = {"lib": df_name}
    top10_dict = {"lib": df_name}
    top25_dict = {"lib": df_name}
    top50_dict = {"lib": df_name}
    
    for c in df_original.columns:
        if c == "fitness" or "_rank" in c:
            continue
        # check if nan is in the column, if so print out the rows with nan
        if df_original[c].isna().sum() > 0:
            print(c)
            print(len(df_original[df_original[c].isna()]))
        # now slice out the rows without nan
        df = df_original[df_original[c].notna()]
        if len(df) == 0:
            rho_dict[c] = np.nan
            ndcg_dict[c] = np.nan
            top10_dict[c] = np.nan
            top25_dict[c] = np.nan
            top50_dict[c] = np.nan
        
        else:

            y_true = df["fitness"].values
            y_score = df[c].values
            if np.all(y_score == y_score[0]):  # Check if all values are identical
                rho_dict[c] = np.nan  # Assign NaN or 0 if needed
            else:
                rho_dict[c] = spearmanr(y_true, y_score).correlation
            ndcg_dict[c] = ndcg_scale(y_true=y_true, y_pred=y_score)
            top10_dict[c] = calc_top_n_percent_recall(y_true=y_true, y_score=y_score, top_n=10)
            top25_dict[c] = calc_top_n_percent_recall(y_true=y_true, y_score=y_score, top_n=25)
            top50_dict[c] = calc_top_n_percent_recall(y_true=y_true, y_score=y_score, top_n=50)

    rho_list.append(rho_dict)
    ndcg_list.append(ndcg_dict)
    top10_list.append(top10_dict)
    top25_list.append(top25_dict)
    top50_list.append(top50_dict)

df_rho = pd.DataFrame(rho_list)
df_ndcg = pd.DataFrame(ndcg_list)
df_top10 = pd.DataFrame(top10_list)
df_top25 = pd.DataFrame(top25_list)
df_top50 = pd.DataFrame(top50_list)

ParLQ
vina_af3-score_seperate-substrate_cofactor-score_only
4
vina_min_af3-score_seperate-substrate_cofactor-score_only
4
vina_max_af3-score_seperate-substrate_cofactor-score_only
4
vina_af3-score_seperate-substrate+carbene_cofactor-score_only
4
vina_min_af3-score_seperate-substrate+carbene_cofactor-score_only
4
vina_max_af3-score_seperate-substrate+carbene_cofactor-score_only
4
vina_af3-score_joint-substrate_cofactor-score_only
10
vina_min_af3-score_joint-substrate_cofactor-score_only
10
vina_max_af3-score_joint-substrate_cofactor-score_only
10
vina_af3-score_joint-substrate+carbene_cofactor-score_only
10
vina_min_af3-score_joint-substrate+carbene_cofactor-score_only
10
vina_max_af3-score_joint-substrate+carbene_cofactor-score_only
10
vina_chai-score_seperate-substrate_cofactor-docked
8
vina_min_chai-score_seperate-substrate_cofactor-docked
8
vina_max_chai-score_seperate-substrate_cofactor-docked
8
vina_chai-score_seperate-substrate_cofactor-score_only
9
vina_min_chai-score_seperate-s

In [23]:
print(list(df_rho.columns))

['lib', 'selectivity', 'n_mut', 'hd', 'ev_score', 'esm_score', 'flowsite_score', 'flowsite_std', 'ligandmpnn_score', 'var_vol', 'substrate_vol', 'cofactor_vol', 'joint_vol', 'esmif_score_apo-score', 'esmif_score_docked-score', 'coves_score_apo_clean-output-100_processed', 'coves_score_sub_clean-output-100_processed', 'Triad_score_score-frompdb-cleanup', 'Triad_score_score-docked', 'ranking_score_agg_score_seperate_af3', 'mean_site_score_agg_score_seperate_af3', 'ptm_agg_score_seperate_af3', 'iptm_agg_score_seperate_af3', 'fraction_disordered_agg_score_seperate_af3', 'chain_ptm_A_agg_score_seperate_af3', 'chain_iptm_A_agg_score_seperate_af3', 'chain_pae_min_AA_agg_score_seperate_af3', 'chain_pae_min_AB_agg_score_seperate_af3', 'chain_iptm_AB_agg_score_seperate_af3', 'chain_pae_min_AC_agg_score_seperate_af3', 'chain_iptm_AC_agg_score_seperate_af3', 'chain_ptm_B_agg_score_seperate_af3', 'chain_iptm_B_agg_score_seperate_af3', 'chain_pae_min_BA_agg_score_seperate_af3', 'chain_iptm_BA_agg_sc

In [21]:
df_rho

,lib,selectivity,n_mut,hd,ev_score,esm_score,flowsite_score,flowsite_std,ligandmpnn_score,var_vol,...,0:C-Si_agg_af3-struct_joint,0:C-Si_avg_af3-struct_joint,0:C-Si_std_af3-struct_joint,0:C-Si_agg_af3-struct_seperate,0:C-Si_avg_af3-struct_seperate,0:C-Si_std_af3-struct_seperate,0:C-Si_avg_chai-struct_joint,0:C-Si_std_chai-struct_joint,0:C-Si_avg_chai-struct_seperate,0:C-Si_std_chai-struct_seperate
0,ParLQ,0.961007,0.231575,-0.231575,0.064764,-0.016563,0.149739,0.767178,0.102997,-0.261154,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,PfTrpB-5iodo,NaN,-0.268085,0.268085,0.419296,0.479928,0.426583,0.234349,0.428536,0.137772,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PfTrpB-5bromo,NaN,-0.331510,0.331510,0.487423,0.523450,0.404282,0.289631,0.462262,0.025118,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,PfTrpB-5cyano,NaN,-0.301809,0.301809,0.396476,0.425708,0.382270,0.268198,0.440015,-0.042512,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,PfTrpB-56chloro,NaN,-0.318860,0.318860,0.388337,0.461043,0.316607,0.348094,0.445602,-0.028684,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,PfTrpB-6chloro,NaN,-0.417893,0.417893,0.569694,0.612678,0.457337,0.241263,0.576349,-0.055604,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Rma-CB,0.643717,-0.061590,0.061590,0.241046,0.214661,0.118630,-0.080166,0.260575,0.214222,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Rma-CSi,0.455375,-0.039558,0.039558,0.117369,0.162468,-0.008253,-0.218604,0.123856,0.150916,...,0.055088,0.24732,-0.093355,0.078906,0.176156,0.110149,-0.011572,0.09156,-0.104207,-0.087255
8,PfTrpB-4cyano,NaN,-0.234953,0.234953,0.329215,0.352757,0.414012,0.121238,0.356667,-0.021313,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,PfTrpB-4bromo,NaN,-0.371285,0.371285,0.474594,0.478416,0.415240,0.212191,0.522690,-0.177474,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Example: Generate a random matrix
matrix = np.random.rand(10, 10)  # 10x10 matrix with values between 0 and 1

# Create the plot
plt.figure(figsize=(8, 6))
plt.imshow(matrix, cmap="viridis", aspect="auto")  # Choose color map (e.g., 'viridis', 'coolwarm')
plt.colorbar(label="Value")  # Add color legend
plt.title("Matrix Heatmap")
plt.xlabel("Columns")
plt.ylabel("Rows")

# Show the plot
plt.show()
